# Hyperparameter Tuning using HyperDrive

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,randint,choice
import os


## Dataset


### Overview

The data contains details of a bank's customers and the target variable is a binary variable reflecting the fact whether the customer left the bank (closed his account) or he continues to be a customer. There are 12 different variables used to predict the churn of a customer, and the data contains around 10,000 such instances.


In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_uri = 'https://raw.githubusercontent.com/himanshu004/AZMLND_Capstone/main/data.csv'
ds = TabularDatasetFactory.from_delimited_files(data_uri)
data = ds.to_pandas_dataframe().dropna()

In [4]:
ws = Workspace.from_config()
experiment_name = 'capstone-hyperdrive2'

experiment=Experiment(ws, experiment_name)

In [5]:
def cluster_validation():
    cluster = 'capstone-cluster'
    try:
        compute_cluster = ComputeTarget(workspace = ws, name = cluster)
        print('Found existing compute cluster!')
    except ComputeTargetException:
        config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2',max_nodes = 4)
        compute_cluster = ComputeTarget.create(ws, cluster, config)
    compute_cluster.wait_for_completion(show_output = True)
    return compute_cluster

In [7]:
#creates new cluster or returns an existing one
compute_cluster = cluster_validation()

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [12]:
#Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1,delay_evaluation = 1)

param_sampling = RandomParameterSampling({
        "--max_iter" : choice(5,10,20,40),
        "--C": uniform(0.5,1.0)
})

estimator = SKLearn(source_directory = "./training",entry_script = 'train.py',compute_target = compute_cluster)

hyperdrive_config = HyperDriveConfig(estimator = estimator,
                                    hyperparameter_sampling = param_sampling,
                                    policy = early_termination_policy,
                                    primary_metric_name = "Accuracy",
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs = 20,
                                    max_concurrent_runs = 4)


In [13]:
hdrun = experiment.submit(config = hyperdrive_config)

## Run Details

In [14]:
RunDetails(hdrun).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

In [17]:
best_run = hdrun.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

for key,value in best_run_metrics.items():
    print(key,' : ',value)

Best Run Id:  HD_c5196eb8-a530-48a2-b7a4-c232936461c1_3

 Accuracy: 0.8035
Regularization Strength:  :  0.5296375858658042
Max iterations:  :  5
Accuracy  :  0.8035


In [18]:
best_saved_model = best_run.register_model(model_name = 'capstone_hyperdrive_best_model.pkl', model_path = './outputs/')

print(best_saved_model)


Model(workspace=Workspace.create(name='quick-starts-ws-136229', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-136229'), name=capstone_hyperdrive_best_model.pkl, id=capstone_hyperdrive_best_model.pkl:1, version=1, tags={}, properties={})
